In [1]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dirPath = r"\data"
fileName = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#fileName = r"reviews_Musical_Instruments_5.json.gz"

path = dirPath + "\\" + fileName

In [4]:
import os
current = os.getcwd()
current
path = current + path
path

'C:\\Study\\Projects\\NLP\\data\\reviews_Tools_and_Home_Improvement_5.json.gz'

In [5]:
dataFile = gzip.open(path, 'r')

In [6]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [7]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [8]:
productReviews[2]

{'reviewerID': 'A37KNOJXE2FU6',
 'asin': '104800001X',
 'reviewerName': 'Joseph Yonke II',
 'helpful': [0, 0],
 'reviewText': 'I wanted something this small to mount on the back of filagree wood piece I cut. They could then be mounted on refrigerators. Works well. Should be able to remove the pieces from the refrig without breaking yet will hold well.',
 'overall': 5.0,
 'summary': 'Very small and thin',
 'unixReviewTime': 1363132800,
 'reviewTime': '03 13, 2013'}

In [9]:
len(productReviews)

134476

In [10]:
#productReviews[101718]

In [11]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [12]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A4IL0CLL27Q33,104800001X,D. Brennan,"[0, 1]","I hate it when my shirt collars, not otherwise...",5.0,Perfect for collar stay management,1390953600,"01 29, 2014"
1,A3Q5W5E7TDVLJF,104800001X,funnyc130,"[0, 0]",These little magnets are really powerful for t...,5.0,Neat,1369958400,"05 31, 2013"


In [13]:
# reduce what we really only need > focus on product id / review
reduceData = productsReviewData.loc[:, ['asin', 'reviewText', 'helpful', 'overall']]
reduceData.shape

(134476, 4)

In [14]:
neutralRating = reduceData.loc[reduceData['overall']==3.0, :]
neutralRating.shape

(10769, 4)

In [15]:
# The index no longer has the right indexes as we remove rating '3' due to neural 
data = notNeutralRating = reduceData.loc[reduceData['overall']!=3.0, :]
data.shape

(123707, 4)

In [16]:
rateTheReview = lambda x: 1.0 if x > 2.0 else 0.0 #Negative/Positive rating 

In [17]:
# Retain the original overall Rating reviews (1 - 5 )
# now we apply 1 for positive review, 0 for negative review
data['trueRating'] = data['overall'].copy().apply(rateTheReview)

In [18]:
# reset the index 
data = data.reset_index()
data.rename(columns={"index": "masterIndex"}, inplace=True)
data.shape

(123707, 6)

In [19]:
data.shape, data.columns

((123707, 6), Index(['masterIndex', 'asin', 'reviewText', 'helpful', 'overall',
        'trueRating'],
       dtype='object'))

In [20]:
data.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating
123705,134474,B00LA96GLC,Easy to wear headlamp that's bright. Good for ...,"[0, 0]",5.0,1.0
123706,134475,B00LA96GLC,Very nice head lamp! Bright led. Fits head wit...,"[0, 0]",5.0,1.0


In [21]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


In [22]:
sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)

'I do not know what to say nay. I have not. shit'

In [23]:
#from ntlk import tokenize
import nltk
import string

In [24]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer() 

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        eachSent = [lemmatizer.lemmatize(word, wordnet.NOUN) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [25]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [26]:
#%%time
data['Processed'] = data.reviewText.apply(cleanUpTextParagraph)


In [27]:
data['CleanedJoin'] = data['Processed'].apply(', '.join)

In [28]:
data.shape, data.columns

((123707, 8),
 Index(['masterIndex', 'asin', 'reviewText', 'helpful', 'overall', 'trueRating',
        'Processed', 'CleanedJoin'],
       dtype='object'))

In [29]:
data.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating,Processed,CleanedJoin
123705,134474,B00LA96GLC,Easy to wear headlamp that's bright. Good for ...,"[0, 0]",5.0,1.0,[easy to wear headlamp that have that be brigh...,easy to wear headlamp that have that be bright...
123706,134475,B00LA96GLC,Very nice head lamp! Bright led. Fits head wit...,"[0, 0]",5.0,1.0,"[very nice head lamp, bright lead, fit head wi...","very nice head lamp, bright lead, fit head wit..."


In [30]:
#data.to_json("cleanData_reviews_Musical_Instruments_5.json.gz", orient='index')

data.drop(columns=['Processed']).to_pickle("preprocessed_"+fileName.split('.')[0] + ".pkl.gz")

In [31]:
#data.sort_values(by='rating', inplace=True)

In [32]:
#sample = reviewTextProcessing['reviewText']
sample = "Bought one and loved it! Ordered another two.One is leaking right after installation!  a lot tries with dollars. Returned it!I thought it was just bad luck because review was so good.One week later, another one leaks when using it (does not leak when not using it).2 leaks out of three. I think it has something to do with QC.It would be 4~5 star if it does not leak...."
cleanUpTextParagraph(sample)

['buy one and love it',
 'order another two one be leak right after installation',
 'a lot try with dollar',
 'return it i think it be just bad luck because review be so good one week later another one leak when use it do not leak when not use it leak out of three',
 'i think it have something to do with qc it would be star if it do not leak']

In [33]:
### Data clean so far